# Homework Assignment 8

In [0]:
!python -m spacy download en_core_web_lg # Important! you need to restart runtime after install

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=eaa8e1b249d3fb9d66561d5e38348c7d1a7d50b61a0e4c2c8384154d38e6b824
  Stored in directory: /tmp/pip-ephem-wheel-cache-o96f2mxf/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


**Connect to Kaggle**

In [0]:
# see more details in the course first lecture
!mkdir /root/.kaggle/
import json
import os

# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"",
"key":""}  # TODO: REMOVE THIS LINE!!!

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


**Question 1:** Select a book or a movie's subtitle, and find the locations that appear in the book/movie. Then, draw these locations on a map (at least 10 locations) (30pt). Connect the discovered locations with lines according to their order of appearance
in the book. For example, if location B appeared immediately after location A, then draw a line connecting locations A and B (20pt).

**Select a book or a movie's subtitle, and find the locations that appear in the book/movie.**

Download the dataset from kaggle

In [0]:
DATASET_NAME = 'tarantino'
KAGGLE_REPO = 'aadityanaik/shakespeareworks'

In [0]:
# searching for the dataset
!rm -r ./datasets/$DATASET_NAME/
!mkdir -p ./datasets/$DATASET_NAME/data
!mkdir -p ./datasets/$DATASET_NAME/zip
!mkdir -p ./datasets/$DATASET_NAME/sql
!ls datasets/$DATASET_NAME/data

!kaggle datasets list -s $DATASET_NAME
# download the dataset from Kaggle and unzip it
!kaggle datasets download $KAGGLE_REPO -p ./datasets/$DATASET_NAME/zip
!unzip ./datasets/$DATASET_NAME/zip/*.zip -d ./datasets/$DATASET_NAME/data
!ls ./datasets/$DATASET_NAME/zip/
!ls ./datasets/$DATASET_NAME/data/
!ls ./datasets/$DATASET_NAME/sql/

rm: cannot remove './datasets/tarantino/': No such file or directory
ref                                                               title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
aadityanaik/shakespeareworks                                      Tarantino Scripts                                 514KB  2018-08-14 18:29:50            137  
fivethirtyeight/cuss-words-and-deaths-in-quentin-tarantino-films  Cuss words and Deaths in Quentin Tarantino Films    8KB  2017-07-19 20:56:53            270  
fivethirtyeight/fivethirtyeight-tarantino-dataset                 FiveThirtyEight Tarantino Dataset                   9KB  2019-04-26 15:06:43             40  
jrw2200/once-upon-a-time-in-hollywood-reddit-thread               Once Upon a Time in Hollywood Reddit                2MB  2019-08-

In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')  # On Colab - Need to restart runtime before import the spacy package
DATASET_PATH = f'./datasets/{DATASET_NAME}/data/'

Find the locations (GPE) using spacy

In [0]:
from tqdm import tqdm
from collections import Counter

def get_locations(txt):
    links_set = set()
    doc = nlp(txt)
    l = []
    i = 0
    for location in doc.ents:
      if location.label_ == "GPE":
        loc_to_add = location.text.lower().strip()
        if i > 0:
          if l[i - 1] == loc_to_add:
            continue
        l.append(loc_to_add)
        i += 1
    return l


Get and Print the list of locations

In [0]:
MOVIE_NAME = 'pulp_fiction_script'
MOVIE_PATH = f'{DATASET_PATH}/{MOVIE_NAME}.txt' 
txt = open(MOVIE_PATH,"r", encoding="utf8", errors="ignore").read()
# MAX_LENGTH = 5000000  
txt = txt.replace("\n", " ")
locations_list = get_locations(txt)
locations_list

['los angeles',
 'denver',
 'hollywood',
 'amsterdam',
 'paris',
 'holland',
 'hollywood',
 'florida',
 'france',
 'jules',
 'b.g.',
 'baldwin park',
 'amsterdam',
 'echo park',
 'mexico',
 'germany',
 'amsterdam',
 'inglewood',
 'amsterdam',
 'su casa',
 'malibu',
 'springfield',
 'marcellus',
 'springfield',
 'l.a.',
 'amsterdam',
 'baton rouge',
 'california',
 'malibu',
 'alhambra',
 'california',
 'texas',
 'hanoi',
 'knoxville',
 'tennessee',
 'paris',
 'hanoi',
 'china',
 'knoxville',
 'tennessee',
 'bora bora',
 'tahiti',
 'mexico',
 'north hollywood',
 'steadicam',
 'marsellus',
 'louisville',
 'marsellus',
 'los angeles',
 'hollywood',
 'jules',
 'toluca lake',
 'the wolf strip',
 'north hollywood',
 'hollywood',
 'b.g.',
 'marsellus']

Find the locations [lat, lon] of each place (if exists) and create dictionary of locations and paths.

In [0]:
import requests
import urllib.parse

locs_dict = {'loc': [], 'lat': [], 'lon': []}
paths_dict = {'loc_start': [], 'lat_start': [], 'lon_start': [],
            'loc_end': [], 'lat_end': [], 'lon_end': []}
prev_name = None
prev_lat = None
prev_lon = None
for loc in locations_list:
  try:
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(loc) +'?format=json'

    response = requests.get(url).json()
    lat = response[0]["lat"]
    lon = response[0]["lon"]
    print(f'Location: {loc}, lat: {lat}, lon: {lon}')
    locs_dict['loc'].append(loc)
    locs_dict['lat'].append(lat)
    locs_dict['lon'].append(lon)

    if prev_name is not None:
      paths_dict['loc_start'].append(prev_name)
      paths_dict['lat_start'].append(prev_lat)
      paths_dict['lon_start'].append(prev_lon)
      paths_dict['loc_end'].append(loc)
      paths_dict['lat_end'].append(lat)
      paths_dict['lon_end'].append(lon)

    prev_name = loc
    prev_lat = response[0]["lat"]
    prev_lon = response[0]["lon"]
  except:
    print(f'Can not find location: {loc}')

Location: los angeles, lat: 34.0536909, lon: -118.2427666
Location: denver, lat: 39.7392364, lon: -104.9848623
Location: hollywood, lat: 34.0980031, lon: -118.3295232
Location: amsterdam, lat: 52.37454030000001, lon: 4.897975505617977
Location: paris, lat: 48.8566969, lon: 2.3514616
Location: holland, lat: 52.5001698, lon: 5.7480821
Location: hollywood, lat: 34.0980031, lon: -118.3295232
Location: florida, lat: 27.7567667, lon: -81.4639835
Location: france, lat: 46.603354, lon: 1.8883335
Location: jules, lat: 45.9514832, lon: -75.8257402
Location: b.g., lat: 64.1011166, lon: -21.7328901
Location: baldwin park, lat: 34.0854739, lon: -117.9611761
Location: amsterdam, lat: 52.37454030000001, lon: 4.897975505617977
Location: echo park, lat: 34.0779819, lon: -118.2567824
Location: mexico, lat: 19.4326296, lon: -99.1331785
Location: germany, lat: 51.0834196, lon: 10.4234469
Location: amsterdam, lat: 52.37454030000001, lon: 4.897975505617977
Location: inglewood, lat: 33.9562003, lon: -118.353

Create dataframes from the dictionaries

In [0]:
df_locations = pd.DataFrame.from_dict(locs_dict)
df_locations

,loc,lat,lon
0,los angeles,34.0536909,-118.2427666
1,denver,39.7392364,-104.9848623
2,hollywood,34.0980031,-118.3295232
3,amsterdam,52.37454030000001,4.897975505617977
4,paris,48.8566969,2.3514616
5,holland,52.5001698,5.7480821
6,hollywood,34.0980031,-118.3295232
7,florida,27.7567667,-81.4639835
8,france,46.603354,1.8883335
9,jules,45.9514832,-75.8257402


In [0]:
df_paths = pd.DataFrame.from_dict(paths_dict)
df_paths

,loc_start,lat_start,lon_start,loc_end,lat_end,lon_end
0,los angeles,34.0536909,-118.2427666,denver,39.7392364,-104.9848623
1,denver,39.7392364,-104.9848623,hollywood,34.0980031,-118.3295232
2,hollywood,34.0980031,-118.3295232,amsterdam,52.37454030000001,4.897975505617977
3,amsterdam,52.37454030000001,4.897975505617977,paris,48.8566969,2.3514616
4,paris,48.8566969,2.3514616,holland,52.5001698,5.7480821
5,holland,52.5001698,5.7480821,hollywood,34.0980031,-118.3295232
6,hollywood,34.0980031,-118.3295232,florida,27.7567667,-81.4639835
7,florida,27.7567667,-81.4639835,france,46.603354,1.8883335
8,france,46.603354,1.8883335,jules,45.9514832,-75.8257402
9,jules,45.9514832,-75.8257402,b.g.,64.1011166,-21.7328901


**Draw these locations on a map (at least 10 locations) (30pt). Connect the discovered locations with lines according to their order of appearance in the book. For example, if location B appeared immediately after location A, then draw a line connecting locations A and B (20pt).**

In [0]:
import plotly.graph_objects as go
import pandas as pd


fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'ISO-3',
    lon = df_locations['lon'],
    lat = df_locations['lat'],
    hoverinfo = 'all',
    text = df_locations['loc'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

flight_paths = []
for i in range(len(df_paths)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'ISO-3',
            lon = [df_paths['lon_start'][i], df_paths['lon_end'][i]],
            lat = [df_paths['lat_start'][i], df_paths['lat_end'][i]],
            mode = 'lines',
            text= ['Source: {} --> Target: {}'.format(df_paths['loc_start'][i], df_paths['loc_end'][i])],
            line = dict(width = 1,color = 'red')
            # opacity = float(df_paths['cnt'][i]) / float(df_paths['cnt'].max()),
        )
    )

fig.update_layout(
    title_text = f'Locations during the movie {MOVIE_NAME} (Path represents that targe location appeared immediately after source location).',
    showlegend = False,
    geo = dict(
        scope = 'world',
        projection_type = 'natural earth',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

**Question 2:** Select a country's' statistic from the [World Development Indicators dataset](https://www.kaggle.com/worldbank/world-development-indicators) (Please notice there are several files in the dataset, such as _Indicators.csv_).
Then, create a choropleth map displaying how the selected statistics changed over time (15pt) 
    
**Bonus:** Create a short animation that displays how the chosen statistics changed over time (15pt)

Download the dataset from Kaggle

In [0]:
DATASET_NAME = 'WDI'
KAGGLE_REPO = 'worldbank/world-development-indicators'

In [0]:
# searching for the dataset
!rm -r ./datasets/$DATASET_NAME/
!mkdir -p ./datasets/$DATASET_NAME/data
!mkdir -p ./datasets/$DATASET_NAME/zip
!mkdir -p ./datasets/$DATASET_NAME/sql
!ls datasets/$DATASET_NAME/data

!kaggle datasets list -s $DATASET_NAME
# download the dataset from Kaggle and unzip it
!kaggle datasets download $KAGGLE_REPO -p ./datasets/$DATASET_NAME/zip
!unzip ./datasets/$DATASET_NAME/zip/*.zip -d ./datasets/$DATASET_NAME/data
!ls ./datasets/$DATASET_NAME/zip/
!ls ./datasets/$DATASET_NAME/data/
!ls ./datasets/$DATASET_NAME/sql/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 51, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 910, in dataset_list_cli
    tag_ids, search, user, mine, page)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 880, in dataset_list
    return [Dataset(d) for d in datasets_list_result]
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 66, in __init__
    self.size = File.get_size(self.totalBytes)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 97, in get_size
    return '%.*f%s' % (precision, size, suffixes[suffix_index])
TypeError: float argument required, not NoneType
 99% 367M/369M [00:03<00:00, 111MB/s] 
100% 369M/369M [00:03<00:00, 103MB/s]
Arch

In [0]:
FILE_PATH = f'./datasets/{DATASET_NAME}/data/Indicators.csv'
geo_data = pd.read_csv(FILE_PATH)
geo_data.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [0]:
Counter(geo_data['CountryName']).most_common()[:10]

[('Mexico', 37244),
 ('Colombia', 37227),
 ('Philippines', 36912),
 ('Peru', 36815),
 ('Costa Rica', 36457),
 ('Thailand', 36355),
 ('Morocco', 36275),
 ('Indonesia', 36252),
 ('Malaysia', 35874),
 ('Turkey', 35819)]

Find intresting indicators

In [0]:
Counter(geo_data['IndicatorCode']).most_common()[:10]

[('SP.POP.TOTL', 13484),
 ('SP.POP.GROW', 13442),
 ('SP.RUR.TOTL.ZS', 13415),
 ('SP.URB.TOTL.IN.ZS', 13415),
 ('SP.RUR.TOTL', 13374),
 ('SP.URB.TOTL', 13374),
 ('SP.URB.GROW', 13337),
 ('AG.SRF.TOTL.K2', 13090),
 ('AG.LND.TOTL.K2', 13086),
 ('SP.RUR.TOTL.ZG', 13061)]

Find the indicator name and the minimum and maximum years

In [0]:
selected_indicator = 'SP.POP.TOTL'
geo_data_ind = geo_data[geo_data['IndicatorCode'] == selected_indicator]
indecator_name = geo_data_ind['IndicatorName'].values[0]
print(f'Indecator Name: {indecator_name}')
min_year = geo_data_ind['Year'].min()
print(f'Minimum Year: {min_year}')
max_year = geo_data_ind['Year'].max()
print(f'Maximum Year: {max_year}')
geo_data_ind.head()

Indecator Name: Population, total
Minimum Year: 1960
Maximum Year: 2014


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
73,Arab World,ARB,"Population, total",SP.POP.TOTL,1960,9.249590e+07
150,Caribbean small states,CSS,"Population, total",SP.POP.TOTL,1960,4.190810e+06
221,Central Europe and the Baltics,CEB,"Population, total",SP.POP.TOTL,1960,9.140158e+07
341,East Asia & Pacific (all income levels),EAS,"Population, total",SP.POP.TOTL,1960,1.042475e+09
462,East Asia & Pacific (developing only),EAP,"Population, total",SP.POP.TOTL,1960,8.964930e+08


**Then, create a choropleth map displaying how the selected statistics changed over time (15pt)** 
    
**Bonus:** Create a short animation that displays how the chosen statistics changed over time (15pt)

**Indicator #1: Short animation that displays how the total opulation changed over time**

In [0]:
import plotly.express as px

# Manipulating the original dataframe
column_of_interst = 'Value'
df_to_plot = geo_data_ind.copy()
df_to_plot = df_to_plot.sort_values('Year', ascending=True)
# Creating the visualization
fig = px.choropleth(df_to_plot, 
                    locations='CountryCode',
                    color=column_of_interst,
                    hover_name='CountryName', 
                    animation_frame="Year",
                    color_continuous_scale='Oranges',
                   )
fig.update_layout(
    title_text = f'World {indecator_name} Over Time Between {min_year}-{max_year}',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()

In [0]:
selected_indicator = 'SP.POP.GROW'
geo_data_ind = geo_data[geo_data['IndicatorCode'] == selected_indicator]
indecator_name = geo_data_ind['IndicatorName'].values[0]
print(f'Indecator Name: {indecator_name}')
min_year = geo_data_ind['Year'].min()
print(f'Minimum Year: {min_year}')
max_year = geo_data_ind['Year'].max()
print(f'Maximum Year: {max_year}')
geo_data_ind.head()

Indecator Name: Population growth (annual %)
Minimum Year: 1960
Maximum Year: 2014


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
3585,Afghanistan,AFG,Population growth (annual %),SP.POP.GROW,1960,1.813677
3637,Albania,ALB,Population growth (annual %),SP.POP.GROW,1960,3.015423
3805,Algeria,DZA,Population growth (annual %),SP.POP.GROW,1960,2.511488
3831,American Samoa,ASM,Population growth (annual %),SP.POP.GROW,1960,1.424246
3843,Andorra,ADO,Population growth (annual %),SP.POP.GROW,1960,7.105041


**Indicator #2: Short animation that displays how the population growth changed over time**

In [0]:
import plotly.express as px

# Manipulating the original dataframe
column_of_interst = 'Value'
df_to_plot = geo_data_ind.copy()
df_to_plot = df_to_plot.sort_values('Year', ascending=True)
# Creating the visualization
fig = px.choropleth(df_to_plot, 
                    locations='CountryCode',
                    color=column_of_interst,
                    hover_name='CountryName', 
                    animation_frame="Year",
                    color_continuous_scale='Oranges',
                   )
fig.update_layout(
    title_text = f'World {indecator_name} Over Time Between {min_year}-{max_year}',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()

In [0]:
selected_indicator = 'SP.DYN.LE00.IN'
geo_data_ind = geo_data[geo_data['IndicatorCode'] == selected_indicator]
indecator_name = geo_data_ind['IndicatorName'].values[0]
print(f'Indecator Name: {indecator_name}')
min_year = geo_data_ind['Year'].min()
print(f'Minimum Year: {min_year}')
max_year = geo_data_ind['Year'].max()
print(f'Maximum Year: {max_year}')
geo_data_ind.head()

Indecator Name: Life expectancy at birth, total (years)
Minimum Year: 1960
Maximum Year: 2013


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
22,Arab World,ARB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,46.847059
103,Caribbean small states,CSS,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,62.271795
188,Central Europe and the Baltics,CEB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,67.823762
282,East Asia & Pacific (all income levels),EAS,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,48.298317
400,East Asia & Pacific (developing only),EAP,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,45.549658


**Indicator #3: Short animation that displays how the life expectancy changed over time**

In [0]:
import plotly.express as px

# Manipulating the original dataframe
column_of_interst = 'Value'
df_to_plot = geo_data_ind.copy()
df_to_plot = df_to_plot.sort_values('Year', ascending=True)
# Creating the visualization
fig = px.choropleth(df_to_plot, 
                    locations='CountryCode',
                    color=column_of_interst,
                    hover_name='CountryName', 
                    animation_frame="Year",
                    color_continuous_scale='Oranges',
                   )
fig.update_layout(
    title_text = f'World {indecator_name} Over Time Between {min_year}-{max_year}',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()

**Question 3:** Select a request type from [San Francisco 311 dataset](https://www.kaggle.com/datasf/case-data-from-san-francisco-311).
Create an interactive map that somehow presents in which areas this request occurs the most (35pt).

Download the dataset from Kaggle

In [0]:
DATASET_NAME = 'sa'
KAGGLE_REPO = 'datasf/case-data-from-san-francisco-311'

In [0]:
# searching for the dataset
!rm -r ./datasets/$DATASET_NAME/
!mkdir -p ./datasets/$DATASET_NAME/data
!mkdir -p ./datasets/$DATASET_NAME/zip
!mkdir -p ./datasets/$DATASET_NAME/sql
!ls datasets/$DATASET_NAME/data

!kaggle datasets list -s $DATASET_NAME
# download the dataset from Kaggle and unzip it
!kaggle datasets download $KAGGLE_REPO -p ./datasets/$DATASET_NAME/zip
!unzip ./datasets/$DATASET_NAME/zip/*.zip -d ./datasets/$DATASET_NAME/data
!ls ./datasets/$DATASET_NAME/zip/
!ls ./datasets/$DATASET_NAME/data/
!ls ./datasets/$DATASET_NAME/sql/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 51, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 910, in dataset_list_cli
    tag_ids, search, user, mine, page)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 880, in dataset_list
    return [Dataset(d) for d in datasets_list_result]
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 66, in __init__
    self.size = File.get_size(self.totalBytes)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 97, in get_size
    return '%.*f%s' % (precision, size, suffixes[suffix_index])
TypeError: float argument required, not NoneType
 98% 120M/122M [00:00<00:00, 151MB/s] 
100% 122M/122M [00:00<00:00, 157MB/s]
Arch

In [0]:
import pandas as pd
FILE_PATH = f'./datasets/{DATASET_NAME}/data/Case_Data_from_San_Francisco_311__SF311.csv'
df_san_fran = pd.read_csv(FILE_PATH, usecols = ['Opened', 'Request Type', 'Point', 'Address'])
df_san_fran.head()

,Opened,Request Type,Address,Point
0,11/30/2008 11:53:00 PM,puc - electric - request_for_service,Intersection of 21ST ST and CAPP ST,"(37.7571008516766, -122.417811874214)"
1,11/30/2008 11:13:00 PM,Illegal Postings - Posting_Too_Large_in_Size,Intersection of BUSH ST and VAN NESS AVE,"(37.7884895281133, -122.421948485141)"
2,11/30/2008 11:07:00 PM,Illegal Postings - Affixed_Improperly,Intersection of EUCLID AVE and MASONIC AVE,"(37.7850837365507, -122.447620029034)"
3,11/30/2008 10:56:00 PM,Sidewalk_Cleaning,"1566 HYDE ST, SAN FRANCISCO, CA, 94109","(37.795328529, -122.418067787)"
4,11/30/2008 10:46:00 PM,Park - Structural_Maintenance,"GGP Panhandle, SAN FRANCISCO, CA, 94117","(37.772204762, -122.4487004)"


Create new columns of lat and lon

In [0]:
def get_lon(x):
  try:
    return float(str(x).split(',')[1].replace(')', '').replace(' ', ''))
  except:
    return None

df_san_fran['lat'] = df_san_fran['Point'].apply(lambda x: float(str(x).split(',')[0].replace('(', '')))
df_san_fran['lon'] = df_san_fran['Point'].apply(lambda x: get_lon(x))
df_san_fran.dropna(inplace=True)
df_san_fran

,Opened,Request Type,Address,Point,lat,lon
0,11/30/2008 11:53:00 PM,puc - electric - request_for_service,Intersection of 21ST ST and CAPP ST,"(37.7571008516766, -122.417811874214)",37.757101,-122.417812
1,11/30/2008 11:13:00 PM,Illegal Postings - Posting_Too_Large_in_Size,Intersection of BUSH ST and VAN NESS AVE,"(37.7884895281133, -122.421948485141)",37.788490,-122.421948
2,11/30/2008 11:07:00 PM,Illegal Postings - Affixed_Improperly,Intersection of EUCLID AVE and MASONIC AVE,"(37.7850837365507, -122.447620029034)",37.785084,-122.447620
3,11/30/2008 10:56:00 PM,Sidewalk_Cleaning,"1566 HYDE ST, SAN FRANCISCO, CA, 94109","(37.795328529, -122.418067787)",37.795329,-122.418068
4,11/30/2008 10:46:00 PM,Park - Structural_Maintenance,"GGP Panhandle, SAN FRANCISCO, CA, 94117","(37.772204762, -122.4487004)",37.772205,-122.448700
...,...,...,...,...,...,...
2022090,01/13/2017 12:57:21 AM,Not_Offensive Graffiti on Private Property,Intersection of HOOKER ALY and MASON ST,"(37.790361207197, -122.410659567458)",37.790361,-122.410660
2022091,01/13/2017 12:57:29 AM,Not_Offensive Graffiti on Public Property,"600 PORTOLA DR, SAN FRANCISCO, CA, 94127","(37.745338725459, -122.452469918715)",37.745339,-122.452470
2022092,01/13/2017 12:58:09 AM,Not_Offensive Graffiti on Public Property,"600 PORTOLA DR, SAN FRANCISCO, CA, 94127","(37.745338725459, -122.452469918715)",37.745339,-122.452470
2022093,01/13/2017 12:58:53 AM,Not_Offensive Graffiti on Public Property,"600 PORTOLA DR, SAN FRANCISCO, CA, 94127","(37.745338725459, -122.452469918715)",37.745339,-122.452470


Find intresting request types

In [0]:
from collections import Counter
Counter(df_san_fran['Request Type']).most_common(10)

[('Bulky Items', 175302),
 ('Sidewalk_Cleaning', 171590),
 ('General Cleaning', 168392),
 ('Not_Offensive Graffiti on Private Property', 107011),
 ('Offensive Graffiti on Public Property', 102991),
 ('Not_Offensive Graffiti on Public Property', 88155),
 ('Hazardous Materials', 77368),
 ('Damaged Parking_Meter', 54581),
 ('Illegal_Dumping', 44562),
 ('Abandoned Vehicle - Car4door', 44383)]

Group by address and request type. The request date is non-relevant here.

In [0]:
df_san_fran_gb = df_san_fran.groupby(['Address', 'lat', 'lon', 'Request Type']).count()
df_san_fran_gb = df_san_fran_gb.reset_index()
df_san_fran_gb

,Address,lat,lon,Request Type,Opened,Point
0,"1 12TH AVE, SAN FRANCISCO, CA, 94118",37.786585,-122.471302,Pavement_Defect,1,1
1,"1 12TH AVE, SAN FRANCISCO, CA, 94118",37.786585,-122.471302,Street_Cleaning,1,1
2,"1 14TH ST, SAN FRANCISCO, CA, 94103",37.768614,-122.413425,Abandoned Vehicle - Car4door,1,1
3,"1 18TH AVE, SAN FRANCISCO, CA, 94121",37.786772,-122.477764,Street_Cleaning,1,1
4,"1 19TH AVE, SAN FRANCISCO, CA, 94121",37.786811,-122.479081,puc - electric - request_for_service,1,1
...,...,...,...,...,...,...
829268,"Youngblood-Coleman Playground, 1398 HUDSON ST,...",37.738768,-122.384502,mons - mons - request_for_service,1,1
829269,"Youngblood-Coleman Playground, 1398 HUDSON ST,...",37.738768,-122.384502,rpd - neighborhood_services - compliment,1,1
829270,"Youngblood-Coleman Playground, 1398 HUDSON ST,...",37.738768,-122.384502,rpd - park_patrol - complaint,1,1
829271,"Youngblood-Coleman Playground, 1398 HUDSON ST,...",37.738768,-122.384502,rpd - park_patrol - request_for_service,1,1


Request Type #1: Number of Offensive Graffiti on Public Property requests in San Francisco by address

In [0]:
request_type = 'Offensive Graffiti on Public Property'
df_san_fran_gb_rt = df_san_fran_gb[df_san_fran_gb['Request Type'] == request_type]
df_san_fran_gb_rt = df_san_fran_gb_rt.sort_values(by=['Point'], ascending=False)
df_san_fran_gb_rt = df_san_fran_gb_rt.head(500)
df_san_fran_gb_rt

,Address,lat,lon,Request Type,Opened,Point
744433,Intersection of HAIGHT ST and MASONIC AVE,37.770210,-122.445345,Offensive Graffiti on Public Property,525,525
733176,Intersection of GEARY BLVD and MASONIC AVE,37.782392,-122.447302,Offensive Graffiti on Public Property,409,409
720256,Intersection of FELL ST and MASONIC AVE,37.772995,-122.445902,Offensive Graffiti on Public Property,346,346
777395,Intersection of MASONIC AVE and OAK ST,37.772080,-122.445717,Offensive Graffiti on Public Property,290,290
695914,Intersection of CHURCH ST and DUBOCE AVE,37.769456,-122.429128,Offensive Graffiti on Public Property,288,288
...,...,...,...,...,...,...
685013,Intersection of BUSH ST and TAYLOR ST,37.789740,-122.412076,Offensive Graffiti on Public Property,35,35
118471,"134 CHURCH ST, SAN FRANCISCO, CA, 94114",37.768909,-122.429128,Offensive Graffiti on Public Property,35,35
633499,Intersection of 2ND AVE and CABRILLO ST,37.775566,-122.459589,Offensive Graffiti on Public Property,35,35
263903,"2249 23RD ST, SAN FRANCISCO, CA, 94107",37.754487,-122.402059,Offensive Graffiti on Public Property,35,35


In [0]:
max_lat = df_san_fran_gb_rt['lat'].max()
min_lat = df_san_fran_gb_rt['lat'].min()
min_lon = df_san_fran_gb_rt['lon'].min()
max_lon = df_san_fran_gb_rt['lon'].max()
print(f'lat {min_lat} , {max_lat}')
print(f'lon {min_lon} , {max_lon}')

lat 37.732467476 , 37.802490183925
lon -122.510341398 , -122.388556249804


**Create an interactive map that somehow presents in which areas this request occurs the most (35pt).**

**Request Type #1:** Number of Offensive Graffiti on Public Property requests in San Francisco by address

In [0]:
import plotly.graph_objects as go

import pandas as pd

df_to_work = df_san_fran_gb_rt.copy()
df_to_work['text'] = df_to_work['Address']
min_value = df_to_work['Point'].min()
max_value = df_to_work['Point'].max()
prev = min_value-1
num_of_cat = 5
jump_val =  int((max_value-min_value)/num_of_cat)
limits = []
for lim in range(min_value, (max_value * 2) ,jump_val):
  if lim == min_value:
    continue
  limits.append((prev+1, lim))
  prev = lim
colors = range(len(limits))
cities = []
scale = 1
eps = 0.01

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_to_work[(df_to_work['Point'] >= lim[0]) & (df_to_work['Point'] <= lim[1])]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Point']*scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = f'Number of {request_type} requests in San Francisco by address',
        showlegend = True,
        geo = go.layout.Geo(
            resolution = 50,
            scope = 'world',
            showframe = False,
            showcoastlines = True,
            landcolor = "rgb(229, 229, 229)",
            countrycolor = "white" ,
            coastlinecolor = "white",
            projection_type = 'hammer',
            lonaxis_range= [ min_lon - eps, max_lon + eps],
            lataxis_range= [min_lat - eps , max_lat + eps ],
            # domain = dict(x = [ 0, 1 ], y = [ 0, 1 ])
        )
    )

fig.show()

Request Type #2: Number of Bulky Items requests in San Francisco by address

In [0]:
request_type = 'Bulky Items'
df_san_fran_gb_rt = df_san_fran_gb[df_san_fran_gb['Request Type'] == request_type]
df_san_fran_gb_rt = df_san_fran_gb_rt.sort_values(by=['Point'], ascending=False)
df_san_fran_gb_rt = df_san_fran_gb_rt.head(500)
df_san_fran_gb_rt

,Address,lat,lon,Request Type,Opened,Point
744885,Intersection of HALE ST and SAN BRUNO AVE,37.732963,-122.405909,Bulky Items,222,222
691624,Intersection of CEDAR ST and LARKIN ST,37.786566,-122.418179,Bulky Items,214,214
749149,Intersection of HIGHLAND AVE and MISSION ST,37.737406,-122.424040,Bulky Items,178,178
630417,Intersection of 26TH ST and FLORIDA ST,37.749544,-122.409762,Bulky Items,171,171
630783,Intersection of 26TH ST and OSAGE ALY,37.749006,-122.418710,Bulky Items,170,170
...,...,...,...,...,...,...
608729,Intersection of 17TH ST and CONNECTICUT ST,37.765072,-122.397737,Bulky Items,30,30
576971,"901 LARKIN ST, SAN FRANCISCO, CA, 94109",37.786109,-122.418142,Bulky Items,30,30
188235,"170 TURK ST, SAN FRANCISCO, CA, 94102",37.783259,-122.412032,Bulky Items,30,30
798768,Intersection of POLK ST and WILLOW ST,37.783568,-122.419273,Bulky Items,30,30


In [0]:
max_lat = df_san_fran_gb_rt['lat'].max()
min_lat = df_san_fran_gb_rt['lat'].min()
min_lon = df_san_fran_gb_rt['lon'].min()
max_lon = df_san_fran_gb_rt['lon'].max()
print(f'lat {min_lat} , {max_lat}')
print(f'lon {min_lon} , {max_lon}')

lat 37.709907440996 , 37.8046840108425
lon -122.508198661991 , -122.386402783965


**Request Type #2:** Number of Bulky Items requests in San Francisco by address

In [0]:
import plotly.graph_objects as go

import pandas as pd

df_to_work = df_san_fran_gb_rt.copy()
df_to_work['text'] = df_to_work['Address']
min_value = df_to_work['Point'].min()
max_value = df_to_work['Point'].max()
prev = min_value-1
num_of_cat = 5
jump_val =  int((max_value-min_value)/num_of_cat)
limits = []
for lim in range(min_value, (max_value * 2) ,jump_val):
  if lim == min_value:
    continue
  limits.append((prev+1, lim))
  prev = lim
colors = range(len(limits))
cities = []
scale = 1
eps = 0.01

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_to_work[(df_to_work['Point'] >= lim[0]) & (df_to_work['Point'] <= lim[1])]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Point']*scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = f'Number of {request_type} requests in San Francisco by address',
        showlegend = True,
        geo = go.layout.Geo(
            resolution = 50,
            scope = 'world',
            showframe = False,
            showcoastlines = True,
            landcolor = "rgb(229, 229, 229)",
            countrycolor = "white" ,
            coastlinecolor = "white",
            projection_type = 'hammer',
            lonaxis_range= [ min_lon - eps, max_lon + eps],
            lataxis_range= [min_lat - eps , max_lat + eps ],
            domain = dict(x = [ 0, 1 ], y = [ 0, 1 ])
        )
    )

fig.show()